In [132]:
def is_prime(n: int) -> bool:
    """
    Return True if `n` is a prime number, otherwise False.

    Uses a simple optimized trial division:
    - handle small numbers directly,
    - eliminate multiples of 2 and 3,
    - then test numbers of the form 6k ± 1 up to sqrt(n).
    """
    
    if n <= 1:
        return False
    if n <= 3:
        return True
    if n % 2 == 0 or n % 3 == 0:
        return False
    i = 5
    while i * i <= n:
        if n % i == 0 or n % (i + 2) == 0:
            return False
        i += 6
    return True

In [133]:
def set_if_inside (matrix: np.ndarray, x: int, y: int, value: int):
    """
    Set `matrix[x, y] = value` if (x, y) is inside the array bounds.

    Parameters
    ----------
    matrix : np.ndarray
        2D grid to modify.
    x, y : int
        Index coordinates.
    value : int
        Value to assign.

    Returns
    -------
    bool
        True if the position was inside and updated, False otherwise.
    """
    
    rows, cols = matrix.shape
    if 0 <= x < rows and  0 <= y < cols:
        matrix[x, y] = value
        return True
    return False

In [134]:
def fill_primes(matrix,offset):
    """
    Fill a 2D numpy array with an Ulam-style prime spiral.

    The spiral starts from the center of the matrix, with the first
    number equal to `offset`. Each subsequent integer is placed while
    walking in an outward spiral pattern.

    Cells are set to:
    - 2 at the center (starting position),
    - 1 where the corresponding number is prime,
    - unchanged (0) otherwise.

    Parameters
    ----------
    matrix : np.ndarray
        2D square array to fill. Initialized with zeros.
    offset : int, optional
        Starting integer for the spiral (default is 1).

    Returns
    -------
    np.ndarray
        The modified matrix containing the prime marking.
    """
    
    rows, cols = matrix.shape

    # Start from the center of the grid
    
    mid = rows // 2
    incr = 1
    side = 0
    current_value = offset
    step_length = 0
    step_increment = 1  # Spiral step growth factor
    x, y = mid,mid

    # Mark the center with a special value (2)
    set_if_inside (matrix,x,y,2)

    # We need to place up to rows * cols numbers
    max_value = rows * cols + offset
    
    while current_value < max_value:
        # Move right and up
        step_length += 1
        for _ in range(step_length):
            x += step_increment
            if is_prime(current_value):
                set_if_inside(matrix, x, y, 1)
            current_value += 1
        for _ in range(step_length):
            y += step_increment
            if is_prime(current_value):
                set_if_inside(matrix, x, y, 1)
            current_value += 1
       
        # Move left and down (step_length increases)
        step_length += 1
        for _ in range(step_length):
            x -= step_increment
            if is_prime(current_value):
                set_if_inside(matrix, x, y, 1)
            current_value += 1
        for _ in range(step_length):
            y -= step_increment
            if is_prime(current_value):
                set_if_inside(matrix, x, y, 1)
            current_value += 1
    
    return matrix

In [154]:
def display_primes(matrix: np.ndarray, offset:int = 1, output: Output | None = None, ) -> np.ndarray:
    """
    Fill and display an Ulam prime spiral.

    The function:
    1. Fills `matrix` with prime indicators using `fill_primes`.
    2. Plots the result with matplotlib, marking:
       - the center (value == 2) in red,
       - prime positions (value == 1) in blue.

    Parameters
    ----------
    matrix : np.ndarray
        2D square array to fill and visualize.
    offset : int, optional
        Starting integer for the spiral (default is 1).
    output : ipywidgets.Output, optional
        Jupyter output widget for displaying/refreshing the plot.
        If None, plotting is done directly.

    Returns
    -------
    np.ndarray
        The filled matrix.
    """
    
    matrix = fill_primes(matrix, offset=offset)

    # Extract coordinates of primes and center
    ys, xs = np.where(matrix == 1)
    y0, x0 = np.where(matrix == 2)
    size = matrix.shape[0]
    
    def _plot() -> None:
        clear_output(wait=True)
        plt.close("all")

        # Scale figure size relative to matrix size (simple heuristic)
        fig_size = max(4, size // 60)
        plt.figure(figsize=(fig_size, fig_size))

        # Center point in red
        if len(x0) > 0:
            plt.scatter(x0, y0, c="red", s=1, label="center")

        # Prime points in blue
        plt.scatter(xs, ys, c="blue", s=1, label="primes")

        plt.xlim(0, size)
        plt.ylim(0, size)
        plt.axis("on")
        plt.grid(False)
        plt.tight_layout()
        plt.show()

    if output is not None:
        with output:
            _plot()
    else:
        _plot()

    return matrix

In [157]:
"""
Interactive prime (Ulam) spiral explorer with FFT and animation.

Requires:
- display_primes(matrix, size, offset)
- fourier_lowpass(matrix, radius)

Both are assumed to be defined elsewhere in your notebook/module.
"""

from __future__ import annotations
import ipywidgets as widgets
from IPython.display import display, clear_output
from ipywidgets import Output
import numpy as np
import matplotlib.pyplot as plt


INITIAL_LIMIT = 100_000
INITIAL_OFFSET = 1
PADDING = 10


output = widgets.Output()

def create_zero_matrix (limit: int, padding: int) -> np.ndarray:
    """
    Create a square zero matrix large enough to hold numbers up to `limit`
    arranged approximately in a sqrt(limit) x sqrt(limit) grid.

    Parameters
    ----------
    limit : int
        Maximum number you want to place in the spiral.
    padding : int
        Extra pixels added to each dimension.

    Returns
    -------
    np.ndarray
        2D zero-initialized matrix.
    """
    size = int(round(limit ** 0.5))
    return np.zeros((size+padding,size+padding))


def build_ulam_spiral_ui (initial_limit:int = INITIAL_LIMIT, initial_offset: int=INITIAL_OFFSET) -> None:

    """
    Build and display an interactive UI for exploring a prime spiral:
    - set limit (how many numbers to check),
    - set offset (starting number),
    - display the prime spiral,
    - run a simple "animation" with changing offsets.
    """

    # Shared state for callbacks
    matrix: np.ndarray | None = None  # will be created on demand

    # --- Widgets ---
    limit_text = widgets.IntText(
        value=initial_limit,
        description='Checks primes up to:',
        placeholder='Enter a number',
    )

    offset_text = widgets.IntText(
        value=initial_offset,
        description='Offset:',
        placeholder='Enter a number',
    )


    button_display = widgets.Button(
        description="Display primes",
        button_style="info", 
        tooltip='Use the number in the text box',
    )


    button_animate = widgets.Button(
        description="Animate",
        tooltip="Display spiral with varying offsets",
    )


    # --- Callback: generate / update prime spiral ---
    def on_display_clicked(_button: widgets.Button) -> None:
        nonlocal matrix
        try:
            limit = limit_text.value
            if limit <=0:
                raise ValueError("Limit must be positive")
            matrix = create_zero_matrix(limit, padding=10)
            size = matrix.shape[0] 
            matrix = display_primes(matrix, offset_text.value, output)
       
        except Exception as exc:
            with output:
                clear_output(wait=True)
                print(f"Error while updating primes:  {exc}")


    def on_animate_clicked(_button: widgets.Button) -> None:
        nonlocal matrix
        try:
            limit = limit_text.value
            matrix = create_zero_matrix(limit, padding=10)
            # Display the map with a changing starting point - as an animation
            size = matrix.shape[0]            
            for j in range(10):
                matrix = create_zero_matrix(limit, padding=10)
                matrix = display_primes(matrix,j*2+1, output)
        except ValueError:
            print("Error " )

    # Bind the function to the button's click event
    button_display.on_click(on_display_clicked)
    button_animate.on_click(on_animate_clicked)


    # --- Show UI ---
    row1 = widgets.HBox([limit_text, offset_text])
    row2 = widgets.HBox([button_display, button_animate])
    display(widgets.VBox([row1, row2, output]))




In [158]:
build_ulam_spiral_ui(1000,1)